YES24 크롤링

In [36]:
import pymysql

In [3]:
# 셀레니움.웹드라이버 준비
from selenium import webdriver

In [6]:
# 크롬드라이브매니저 설치
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

'/Users/hotbari/.wdm/drivers/chromedriver/mac64/121.0.6167.85/chromedriver-mac-x64/chromedriver'

In [56]:
# 크롬 창 열림
browser = webdriver.Chrome()

In [57]:
# 원하는 웹사이트(URL)의 데이터 불러오기
# line13의 크롬창을 닫고 실행하면 오류 발생
url = 'https://www.yes24.com/Product/category/bestseller?CategoryNumber=001&sumgb=06'
browser.get(url)

In [59]:
# 클래스명으로 요소 불러오기
from selenium.webdriver.common.by import By
# browser.find_element(By.CLASS_NAME, 'uU7dJb').text

In [60]:
# 1페이지 링크데이터 전부 수집
datas = browser.find_elements(By.CLASS_NAME,'gd_name')
for i in datas:
    print(i.get_attribute('href'))


https://www.yes24.com/Product/Goods/122120495
https://www.yes24.com/Product/Goods/123675187
https://www.yes24.com/Product/Goods/117014613
https://www.yes24.com/Product/Goods/124702623
https://www.yes24.com/Product/Goods/124043812
https://www.yes24.com/Product/Goods/124043802
https://www.yes24.com/Product/Goods/123400303
https://www.yes24.com/Product/Goods/122090360
https://www.yes24.com/Product/Goods/122944685
https://www.yes24.com/Product/Goods/118579613
https://www.yes24.com/Product/Goods/124539805
https://www.yes24.com/Product/Goods/124110821
https://www.yes24.com/Product/Goods/118413014
https://www.yes24.com/Product/Goods/122428190
https://www.yes24.com/Product/Goods/124112599
https://www.yes24.com/Product/Goods/124644663
https://www.yes24.com/Product/Goods/124702605
https://www.yes24.com/Product/Goods/119458052
https://www.yes24.com/Product/Goods/124849202
https://www.yes24.com/Product/Goods/119562091
https://www.yes24.com/Product/Goods/124644681
https://www.yes24.com/Product/Good

In [61]:
# 속도 제어
import time

# 3페이지까지 가져오기
link_list = []
for i in range(1,4) :
    print(f"현재 {i} 페이지 링크데이터 수집 중")
    url = f"https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24"
    browser.get(url)

    datas = browser.find_elements(By.CLASS_NAME,'gd_name')

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)

    time.sleep(2)

    

현재 1 페이지 링크데이터 수집 중
현재 2 페이지 링크데이터 수집 중
현재 3 페이지 링크데이터 수집 중


In [70]:
# 데이터 베이스 연동 후 수집한 데이터를 DB에 저장

import pymysql
import time
import re 
from datetime import datetime

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='010913',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur :

    for link in link_list:
        browser.get(link)

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
        
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일')
        if match :
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")

        else :
            publishing = "2023-02-06"

        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
        
        review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
        review = int(review.replace(",",""))
        
        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        salse = int(sales.replace(",",""))

        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",",""))

        full_text = browser.find_element(By.CLASS_NAME, 'gd_best.gd_best_tp02').text
        parts = full_text.split(" | ")
        try:
            ranking_part = parts[0]
            ranking = ''.join(filer(str.isdigit, ranking_part))

        except:
            ranking = 0

    
        try:
            ranking_weeks_part = parts[0]
            ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))

        except :
            ranking_weeks = 0
    
        
        ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[2][:-1]
    
        sql = "INSET INTO Books (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks) VALUES( %s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()

        time.sleep(2)


    

TypeError: search() missing 1 required positional argument: 'string'